## Importing python packages

In [112]:
from matplotlib import pyplot as plt                                       
import numpy as np                                                        
import pandas as pd
import xarray as xr
import cartopy
import dask
from tqdm.autonotebook import tqdm 
import intake
import fsspec

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina'

## Loading data from google cloud

In [113]:
url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)

## Defining functions for preprocess and analysis

In [187]:
def drop_all_bounds(ds):
    """Drop coordinates like 'time_bounds' from datasets,
    which can lead to issues when merging."""
    drop_vars = [vname for vname in ds.coords
                 if (('_bounds') in vname ) or ('_bnds') in vname]
    return ds.drop(drop_vars)
def get_lat_name(ds):
    """Figure out what is the latitude coordinate for each dataset."""
    for lat_name in ['lat', 'latitude']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    """Return global mean of a whole dataset."""
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(ds.dims) - {'time'}
    return (ds * weight).mean(other_dims)

## Selecting the required experiments and variables

In [115]:
# load a few models to illustrate the problem
query = dict(experiment_id=['abrupt-4xCO2','piControl'], table_id='Amon',
             variable_id=['tas', 'rsut','rsdt','rlut']
            )
cat = col.search(require_all_on=["source_id"],**query)

cat.df['source_id'].unique()
z_kwargs = {'consolidated': True, 'decode_times':False}
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dset_dict = cat.to_dataset_dict(zarr_kwargs=z_kwargs,preprocess=drop_all_bounds)#


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


## Making a dictonary to store data wrt experiment and model name

In [116]:
from collections import defaultdict
ab = defaultdict(dict)
for k, ds in dset_dict.items():
    exp=k.split('.')[-3]
    mod=k.split('.')[-4]
    ab[mod][exp]=ds

## Computing for all models

In [122]:
ab_ = dask.compute(ab)[0]

## concatenate the experiments together into a single Dataset

In [128]:
expts = ['piControl', 'abrupt-4xCO2']
expt_da = xr.DataArray(expts, dims='experiment_id',
                       coords={'experiment_id': expts})

dsets_aligned = {}

for k, v in tqdm(ab_.items()):
    if k in ['IPSL-CM6A-LR','MRI-ESM2-0','EC-Earth3']: #Skipped these experiments because there are some problem with the time coordinate
        continue
    expt_dsets = v.values()
    if any([d is None for d in expt_dsets]):
        print(f"Missing experiment for {k}")
        continue

    for ds in expt_dsets:
        ds=xr.decode_cf(ds)
        ds.coords['year'] = ds.time.dt.year - ds.time.dt.year[0]

    
    dsets_ann_mean = [v[expt].pipe(global_mean)
                             .swap_dims({'time': 'year'})
                             .drop('time')
                             .coarsen(year=12).mean()
                      for expt in expts]
    for d in dsets_ann_mean:
        d['year']=range(d.year.shape[0])

    # align everything with the 4xCO2 experiment
    dsets_aligned[k] = xr.concat(dsets_ann_mean, join='right',
                                 dim=expt_da)

C:\Users\ligin\anaconda3\lib\site-packages\xarray\coding\times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\Users\ligin\anaconda3\lib\site-packages\numpy\core\_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
C:\Users\ligin\anaconda3\lib\site-packages\xarray\coding\times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\Users\ligin\anaconda3\lib\site-packages\numpy\core\_asarray.py:85: SerializationWarning: Unable t

C:\Users\ligin\anaconda3\lib\site-packages\numpy\core\_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


In [130]:
dsets_aligned_ = dask.compute(dsets_aligned)[0]

C:\Users\ligin\anaconda3\lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


## concatenate across models to produce one big dataset with all the required variables

In [136]:
source_ids = list(dsets_aligned_.keys())
source_da = xr.DataArray(source_ids, dims='source_id',
                         coords={'source_id': source_ids})

big_ds = xr.concat([ds.reset_coords(drop=True)
                    for ds in dsets_aligned_.values()],
                   dim=source_da)
big_ds

<xarray.Dataset>
Dimensions:        (experiment_id: 2, source_id: 42, year: 1000)
Coordinates:
  * source_id      (source_id) object 'IITM-ESM' 'TaiESM1' ... 'GISS-E2-1-G'
  * year           (year) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * experiment_id  (experiment_id) object 'piControl' 'abrupt-4xCO2'
Data variables:
    rlut           (source_id, experiment_id, year) float64 233.6 233.5 ... nan
    rsdt           (source_id, experiment_id, year) float64 338.8 338.8 ... nan
    rsut           (source_id, experiment_id, year) float64 104.6 104.6 ... nan
    tas            (source_id, experiment_id, year) float64 287.1 287.1 ... nan
    None           (source_id) <U15 'IITM-ESM' 'TaiESM1' ... 'GISS-E2-1-G'

## Calculation of ECS

###   
         ------------------------------------------------------------------------------
        #   Compute Equilibrium Climate Sensitivity (ECS)
        #   Approach: Gregory et al. (2004; https://doi.org/10.1029/2003GL018747)
        #   Note: We don't use rtmt (TOA net radiative flux) because it's available for 
        #   fewer models. We only use years 20 to 150 as the resulting ECS values agree
        #   better with slab models and long simulations (Dunne et al., 2020). Equation
        #   for computing regression line: 
        #   https://www4.stat.ncsu.edu/~dickey/summer_institute/formulas
        #------------------------------------------------------------------------------

In [170]:
year_sel = slice(20, 150) # Year range
ds_abrupt = big_ds.sel(year=year_sel, experiment_id='abrupt-4xCO2').reset_coords(drop=True) #selecting abrupt-4xCO2 experiment
ds_pi = big_ds.sel(year=year_sel, experiment_id='piControl').reset_coords(drop=True)  #selecting piControl experiment

In [174]:
deltaN = (ds_abrupt.rsdt-ds_abrupt.rsut-ds_abrupt.rlut)-(ds_pi.rsdt-ds_pi.rsut-ds_pi.rlut) # Top of atmosphere (or model) net radiative flux anomaly (W/m2)
deltaT = ds_abrupt.tas-ds_pi.tas # Temperature anomaly (degrees C)

In [183]:
Y, X = deltaN, deltaT          
slope = np.sum((Y-np.mean(Y,axis=1))*(X-np.mean(X,axis=1)),axis=1)/np.sum((X-np.mean(X,axis=1))**2,axis=1)
intercept = np.mean(Y,axis=1)-slope*np.mean(X,axis=1)                
# Compute ECS
ecs = (-intercept/slope)/2

In [186]:
ecs.to_netcdf(path='CMIP6_ecs.nc') #saving as netcdf file